# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [4]:
len(train_doc)

663

In [5]:
len(val_doc)

100

## Preprocessing

In [6]:
def normalize_image1(data):
    return data/127.5-1

def normalize_image2(data):
    return data/255

def normalize_image3(x):
    return (x - x.min()) / (x.max() - x.min())

# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

def add_gaussian_noise(image):
    row,col,ch= image.shape
    mean = 0
    #var = 0.1
    #sigma = var**0.5
    gauss = np.random.normal(mean,1,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
def generator(source_path, folder_list, batch_size, isValidation=False):
    
    x = 30 # number of frames
    y = 120 # image width
    z = 120 # image height
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            # We will also build a agumented batch data
            batch_data_aug = np.zeros((batch_size,x,y,z,3))
            batch_labels_aug = np.zeros((batch_size,5))
            
            #Horizontal flip
            batch_data_flip = np.zeros((batch_size,x,y,z,3))
            batch_labels_flip = np.zeros((batch_size,5))
            
            #Gaussian noise
            batch_data_noise = np.zeros((batch_size,x,y,z,3))
            batch_labels_noise = np.zeros((batch_size,5))
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                   
                    # Cropping non symmetric frames
                    if image.shape[0] != image.shape[1]:
                        image=image[0:120,20:140]
                    
                    resized = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)
                    
                    normalized = normalize_image3(resized)
                    batch_data[folder,idx] = normalized
                    
                    #Affine transformation
                    batch_data_aug[folder,idx] = cv2.warpAffine(normalized, M, (normalized.shape[0], normalized.shape[1]))
                    
                    # Data with horizontal flip
                    batch_data_flip[folder,idx]= np.flip(resized,1)
                    
                    #Add gaussian noise
                    batch_data_noise[folder,idx]=add_gaussian_noise(resized)
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_noise[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
                # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
                if int(t[folder + (batch*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
                elif int(t[folder + (batch*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
                else:
                    batch_labels_flip[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
            
            batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
            batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)
            batch_data_final = np.append(batch_data_final, batch_data_noise, axis = 0)
            
            batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0)
            batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
            batch_labels_final = np.append(batch_labels_final, batch_labels_noise, axis = 0)
            
            if isValidation:
                batch_data_final=batch_data
                batch_labels_final= batch_labels

            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            #Agumented data
            batch_data_aug = np.zeros((batch_size,x,y,z,3))
            batch_labels_aug = np.zeros((batch_size,5))
            
            #Horizontal flip
            batch_data_flip = np.zeros((batch_size,x,y,z,3))
            batch_labels_flip = np.zeros((batch_size,5))
            
            #Gaussian noise
            batch_data_noise = np.zeros((batch_size,x,y,z,3))
            batch_labels_noise = np.zeros((batch_size,5))
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resized = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)
                    normalized = normalize_image3(resized)
                    batch_data[folder,idx] = normalized
                    
                    #Affine transformation
                    batch_data_aug[folder,idx] = cv2.warpAffine(normalized, M, (normalized.shape[0], normalized.shape[1]))
                    
                    # Data with horizontal flip
                    batch_data_flip[folder,idx]= np.flip(resized,1)
                    
                    #Add gaussian noise
                    batch_data_noise[folder,idx]=add_gaussian_noise(resized)
                    
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    batch_labels_noise[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
                    # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
                    if int(t[folder + (batch*batch_size)].strip().split(';')[2])==0:
                        batch_labels_flip[folder, 1] = 1
                    elif int(t[folder + (batch*batch_size)].strip().split(';')[2])==1:
                        batch_labels_flip[folder, 0] = 1
                    
                    else:
                        batch_labels_flip[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
                    
            batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
            batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)
            batch_data_final = np.append(batch_data_final, batch_data_noise, axis = 0)
            
            batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0)
            batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
            batch_labels_final = np.append(batch_labels_final, batch_labels_noise, axis = 0)
            
            if isValidation:
                batch_data_final=batch_data
                batch_labels_final= batch_labels
                
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
nb_filters = [16,32,64,128]
nb_dense = [256, 128, 5]
input_shape = (30, 120, 120, 3)

## Model 1

In [10]:
import keras
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Conv2D, Dropout, GRU
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
# create a VGG16 "model", we will use
# image with shape (224, 224, 3)
vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(120, 120, 3)
)
# do not train first layers, I want to only train
# the 4 last layers (my own choice, up to you)
#for layer in vgg.layers[:-4]:
#    layer.trainable = False

# create a Sequential model
model1 = Sequential()
# add vgg model for 5 input images (keeping the right shape
model1.add(
    TimeDistributed(vgg, input_shape=(30, 120, 120, 3))
)
# now, flatten on each output to send 5 
# outputs with one dimension to LSTM
model1.add(
    TimeDistributed(
        Flatten()
    )
)
model1.add(LSTM(256, activation='relu', return_sequences=False))
# finalize with standard Dense, Dropout...
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(.3))
model1.add(Dense(5, activation='softmax'))

Using TensorFlow backend.


## Model 1.1

In [11]:
#import keras
#from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Conv2D, Dropout, GRU
#from keras import Sequential
#from keras.applications.vgg16 import VGG16
#from keras import optimizers
#from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
## create a VGG16 "model", we will use
## image with shape (224, 224, 3)
#vgg = VGG16(
#    include_top=False,
#    weights='imagenet',
#    input_shape=(120, 120, 3)
#)
## do not train first layers, I want to only train
## the 4 last layers (my own choice, up to you)
##for layer in vgg.layers[:-4]:
##    layer.trainable = False
#
## create a Sequential model
#model11 = Sequential()
## add vgg model for 5 input images (keeping the right shape
#model11.add(
#    TimeDistributed(vgg, input_shape=(30, 120, 120, 3))
#)
## now, flatten on each output to send 5 
## outputs with one dimension to LSTM
#model11.add(
#    TimeDistributed(
#        Flatten()
#    )
#)
#model11.add(GRU(256, activation='relu', return_sequences=False))
## finalize with standard Dense, Dropout...
#model11.add(Dense(64, activation='relu'))
#model11.add(Dropout(.3))
#model11.add(Dense(5, activation='softmax'))

## Model 2

In [12]:
frames=30
channels=3
rows=120
columns=120
classes=5

In [13]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam

video = Input(shape=(frames, rows, columns, channels))

cnn_base = VGG16(input_shape=(rows, columns, channels), weights="imagenet", include_top=False)

cnn_out = GlobalAveragePooling2D()(cnn_base.output)

cnn = Model(input=cnn_base.input, output=cnn_out)
cnn.trainable = False

encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = LSTM(256)(encoded_frames)

hidden_layer = Dense(output_dim=1024, activation="relu")(encoded_sequence)
outputs = Dense(output_dim=classes, activation="softmax")(hidden_layer)

model2 = Model([video], outputs)

optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

model2.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["categorical_accuracy"]) 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024)`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5)`


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

## Model 2.1

In [14]:
#from keras.applications.vgg16 import VGG16
#from keras.models import Model
#from keras.layers import Dense, Input
#from keras.layers.pooling import GlobalAveragePooling2D
#from keras.layers.recurrent import LSTM
#from keras.layers.wrappers import TimeDistributed
#from keras.optimizers import Nadam
#
#video1 = Input(shape=(frames, rows, columns, channels))
#
#cnn_base1 = VGG16(input_shape=(rows, columns, channels), weights="imagenet", include_top=False)
#
#cnn_out1 = GlobalAveragePooling2D()(cnn_base1.output)
#
#cnn1 = Model(input=cnn_base1.input, output=cnn_out1)
#cnn1.trainable = False
#
#encoded_frames1 = TimeDistributed(cnn1)(video1)
#encoded_sequence1 = GRU(256)(encoded_frames1)
#
#hidden_layer1 = Dense(output_dim=1024, activation="relu")(encoded_sequence1)
#outputs1 = Dense(output_dim=classes, activation="softmax")(hidden_layer1)
#
#model21 = Model([video1], outputs1)
#
#optimizer1 = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
#
#model21.compile(loss="categorical_crossentropy", optimizer=optimizer1,metrics=["categorical_accuracy"]) 

In [15]:
rate = 0.001
optimiser = optimizers.Adam() #write your optimizer
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())
#print (model11.summary())
print (model2.summary())
#print (model21.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 4608)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               4981760   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 19,713,221
Trainable params: 19,713,221
Non-trainable params: 0
________________________________________________________________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size, isValidation=True)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
batch_size = 10
num_epochs = 10
history1 = model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10


InternalError: Dst tensor is not initialized.
	 [[Node: _arg_time_distributed_1_input_0_0/_563 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_3266__arg_time_distributed_1_input_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: metrics_1/categorical_accuracy/Mean/_597 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3284_metrics_1/categorical_accuracy/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
batch_size = 10
num_epochs = 10
history2 = model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
#batch_size = 10
#num_epochs = 10
#history11 = model11.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
#batch_size = 10
#num_epochs = 10
#history21 = model21.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
print(history1.history.keys())

In [ ]:
print(history2.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history1.history['categorical_accuracy'])
plt.plot(history1.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history2.history['categorical_accuracy'])
plt.plot(history2.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()